# InsightML

## Library

In [ ]:
import csv
import itertools
import json
import numpy as np
import pandas as pd
import pandas_flavor as pf

## Magic Format

In [ ]:

def about_my_csv(filename):
    # Ouvrir le fichier en mode lecture et détecter le format de délimitation de colonne
    with open(filename, 'r', newline='') as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.read(1024))
    return dialect

def about_my_xls(filename):
    # Lire le fichier Excel en utilisant le module openpyxl et sans en-têtes de colonne
    df_temp = pd.read_excel(filename, engine='openpyxl', header=None)
    # Détecter le format de délimitation de colonne à l'aide de la fonction csv.Sniffer().sniff()
    dialect = csv.Sniffer().sniff(df_temp.to_csv(index=False, header=False))
    return dialect

def about_my_json(filename):
    # Lire le fichier Json pour récuperer la donnée
    with open(filename, 'r') as f:
        data = f.read()
    # Détecter le format du fichier JSON
    try:
        json.loads(data)
        orient = 'records'
    except ValueError:
        orient = 'columns'
    return orient

def about_my_h5(filename):
    # Lire les clés disponibles dans le fichier HDF5
    with pd.HDFStore(filename, mode='r') as store:
        keys = store.keys()

    # Trouver la clé correspondant au plus grand ensemble de données
    max_size = 0
    max_key = None
    for key in keys:
        size = pd.read_hdf(filename, key=key, stop=0).memory_usage(index=True, deep=True).sum()
        if size > max_size:
            max_size = size
            max_key = key
    return max_key


## Importation

In [ ]:

path_to_file = 'chemin/vers/fichier.csv'

extention = path_to_file.split('.')[-1]

if extention == 'csv':
    # Infos format about my csv.
    my_dialect = about_my_csv(path_to_file)
    # Make dataframe.
    df = pd.read_csv(path_to_file, dialect=my_dialect)

elif extention in ['xls', 'xlsx','xlsm','xlsb']:
    # Infos format about my excel.
    my_dialect = about_my_csv(path_to_file)
    # Make dataframe.
    df = pd.read_excel(path_to_file, engine='openpyxl', delimiter=my_dialect.delimiter)

elif extention == 'json':
    # Infos format about my json.
    my_orient = about_my_csv(path_to_file)
    # Make dataframe.
    df = pd.read_json(path_to_file, orient=my_orient)

elif extention == 'h5':
    # Infos format about my json.
    my_key = about_my_h5(path_to_file)
    # Make dataframe
    df = pd.read_hdf(path_to_file, key=my_key, mode='r')

else: 
    print('format non pris en charge')
    print('Option à venir : SQL, Parquet, Feather, Pickle, HTML, XML')

## Supp Colonnes

In [ ]:
# Calcul du pourcentage de lignes vides pour chaque colonne
pourcentage_lignes_vides = df.isna().sum() / len(df)

# Sélection des colonnes à conserver (celles ayant moins de 30% de lignes vides)
colonnes_a_conserver = pourcentage_lignes_vides[pourcentage_lignes_vides <= 0.3].index

# Création d'un nouveau DataFrame ne contenant que les colonnes à conserver
df = df[colonnes_a_conserver]

## Supp lignes

In [ ]:
## Supprime les lignes avec des données sporatiques
# Calculer le pourcentage de valeurs manquantes dans chaque ligne
missing_pct = df.isnull().sum(axis=1) / df.shape[1]
# Calculer le seuil de tolérance en nombre de colonnes sans valeurs
threshold = 0.15 * df.shape[1]
# Supprimer les lignes dont le nombre de colonnes sans valeurs est supérieur au seuil de tolérance
df = df.dropna(thresh=df.shape[1]-threshold)

## DateTime

In [ ]:
# Définir la méthode 'detect_date' pour détecter les colonnes de texte contenant des dates
@pf.register_dataframe_method
def detect_date(df):
    for col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass
    return df

# Détecter et convertir les colonnes de texte en dates
df = df.detect_date()

In [ ]:
# Identifier les colonnes de type datetime
date_cols = [col for col in df.columns if df[col].dtype == 'datetime64[ns]']

# Vérifier s'il y a au moins deux colonnes de date
if len(date_cols) < 2:
    raise ValueError('Il n\'y a pas assez de colonnes de date pour faire des différences de temps.')
else:
    # Normaliser les dates sur UTC
    for date_col in date_cols:
        df[date_col] = df[date_col].dt.tz_convert('Etc/UTC')

    # Créer toutes les combinaisons de colonnes de date
    list_date_cols = list(itertools.combinations(date_cols, 2))

    # Calculer la différence de temps entre chaque paire de colonnes de date
    for paire_date in list_date_cols:
        first_date = paire_date[0]
        second_date = paire_date[1]

        df[f'{first_date}_x_{second_date}'] = (df[second_date] - df[first_date]).dt.total_seconds()


# Extraire : Year, Day, WeekDay, Hour, Minute, Second, Microsecond
for date_col in date_cols:
    # Ajouter les colonnes pour chaque attribut de date
    df[f'{date_col}_year'] = df[date_col].dt.year
    df[f'{date_col}_day_of_year'] = df[date_col].dt.dayofyear
    df[f'{date_col}_day_of_week'] = df[date_col].dt.dayofweek
    df[f'{date_col}_hour'] = df[date_col].dt.hour
    df[f'{date_col}_minute'] = df[date_col].dt.minute
    df[f'{date_col}_second'] = df[date_col].dt.second
    df[f'{date_col}_microsecond'] = df[date_col].dt.microsecond

    df.drop(date_col, axis=1, inplace=True)

## Numerical

In [ ]:

numeric_features = df.select_dtypes([np.number]).columns

for numerical in numeric_features:
    # Garde uniquement les valeurs qui sont dans l'intervalle [moyenne-2*ecart_type, moyenne+2*ecart_type]
    to_keep = (df[numerical] > df[numerical].mean() - 2*df[numerical].std()) & (df[numerical] < df[numerical].mean() + 2*df[numerical].std())
    df = df.loc[to_keep,:]

    # Transforme en catégoriel quand moins de 8 valeurs différentes.
    if df[numerical].value_counts() <= 8:
        df[numerical] = df[numerical].astype(str)

    # Supression valeurs manquantes au dessus de 10 %
    pourcentage_valeur_manquante = 100*df[numerical].isnull().sum()/len(df)
    if pourcentage_valeur_manquante >= 10:
        df.drop(numerical, axis=1, inplace=True)

## Categoriel

In [ ]:
categorical_features = df.select_dtypes("object").columns

for categorical in categorical_features:
    # Remplacement des valeurs nulles par la valeur "manquante"
    df[categorical] = df[categorical].fillna('manquante')

    ## Remplace les valeurs sporatique par la valeur "autre"
    # Calcul du pourcentage de chaque valeur dans la colonne
    counts = df[categorical].value_counts(normalize=True)
    # Sélection des valeurs qui représentent moins de 10% de la colonne
    mask = (counts < 0.1)
    # Remplacement des valeurs sélectionnées par la valeur "autre"
    df[categorical] = df[categorical].replace(counts[mask].index.tolist(), 'autre')

    # Suppression des colonnes contenant une seule valeur
    if df[categorical].nunique() == 1:
        df.drop(categorical, axis=1, inplace=True)


#### Par la suite
* Détecter format carte bancaire, indicatif telephone...
* Convertir adresse en format lat.nlong
* Convertir les listes de mots en catégories quand c'est possible.
* ...

## Séparation en variables explicative et cible

In [ ]:
target_name = df.columns[-1]

Y = df[:][target_name]
X = df.drop(columns=[target_name])

## Suppr colonnes ultra corrélées

In [ ]:
corr = X.corr()

high_corr_list = []
cols = corr.columns

for j in cols:
    for i, item in corr[j].iteritems():
        if (i!=j) and abs(item) > 0.9:
            high_corr_list.append((i,j))
high_corr_list

no_keep = [high_corr_list[i][0] for i in range(len(high_corr_list)) if i%2 == 0]

columns_to_keep = [c for c in X.columns if c not in no_keep]

X_clean = X.loc[:, columns_to_keep]
X_clean.columns

### Regression 

#### Séparation variable explicative et target 
X
Y

#### Séparation en Entrainement & Test
X_train X_test 
!!! Stratify si Y valeur catégoriel 

#### Régler déséquilibre dataset
pour une différence supérieur à 10%, 
à noté par la suite qu'une meilleur approche est de faire attention :
à la précision, le rappel, la F1-score et l'aire sous la courbe ROC. 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rUs=RandomUnderSampler()
X_ru, y_ru = rUs.fit_resample(X_train,y_train)

#### Pipeline 
numeric_features, categorical_features, preprocessor, StandardScaler....

#### Training Model 
Logistic Regression
Linear Regression
Random Forest Classifier
Random Forest Classifier 

### Result 

#### Qualitatif (Classification)
Accuracy 

#### Quantitatif (Numérique)
R2

#### Grid search : Random Forest Classifier
Accuracy
F1

#### Grid search : Random Forest Regressor 
print('MAE: ', mean_absolute_error(y_test, y_pred))
print('MSE: ', mean_squared_error(y_test, y_pred)) 

## Affichage